# Pacotes

In [215]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [216]:
import pandas as pd
import glob
import os
 
import openpyxl

# 1) Coleta dos dados

### 1.1) Leitura dos arquivos

#### OBS: Escolhi utilizar os arquivos csv pela facilidade de leitura e poder utilizar chunks de dados

In [217]:
def get_csv_files(years: range = range(2000, 2001), directory: str = "../data", chunksize: int = 10000) -> list: 
    dataframes_list = []

    for year in years:
        file_name = f"GERACAO_USINA_{year}.csv"
        file_path = os.path.join(directory, file_name)

        print(f"\nLendo o arquivo {file_name}...")

        if glob.glob(file_path):
            # Leia o arquivo XLSX
            df = pd.read_csv(file_path, chunksize=chunksize, delimiter=";", encoding="utf-8", low_memory=False)

            # Adicione o DataFrame à lista
            dataframes_list.append(df)
        else:
            raise Exception(f"O arquivo {file_name} não foi encontrado.")
            

    return dataframes_list

In [218]:
try: 
    dataframes = get_csv_files(range(2000, 2021), "../data", 10000)
except Exception as e:
    print(f"Erro: {e}")


Lendo o arquivo GERACAO_USINA_2000.csv...

Lendo o arquivo GERACAO_USINA_2001.csv...

Lendo o arquivo GERACAO_USINA_2002.csv...

Lendo o arquivo GERACAO_USINA_2003.csv...

Lendo o arquivo GERACAO_USINA_2004.csv...

Lendo o arquivo GERACAO_USINA_2005.csv...

Lendo o arquivo GERACAO_USINA_2006.csv...

Lendo o arquivo GERACAO_USINA_2007.csv...

Lendo o arquivo GERACAO_USINA_2008.csv...

Lendo o arquivo GERACAO_USINA_2009.csv...

Lendo o arquivo GERACAO_USINA_2010.csv...

Lendo o arquivo GERACAO_USINA_2011.csv...

Lendo o arquivo GERACAO_USINA_2012.csv...

Lendo o arquivo GERACAO_USINA_2013.csv...

Lendo o arquivo GERACAO_USINA_2014.csv...

Lendo o arquivo GERACAO_USINA_2015.csv...

Lendo o arquivo GERACAO_USINA_2016.csv...

Lendo o arquivo GERACAO_USINA_2017.csv...

Lendo o arquivo GERACAO_USINA_2018.csv...

Lendo o arquivo GERACAO_USINA_2019.csv...

Lendo o arquivo GERACAO_USINA_2020.csv...


### 1.2) Lendo Dataframes por chunks

In [219]:
def reading_chunks(df, cols: list = ["din_instante", "id_subsistema", "val_geracao", "nom_tipousina"]):
    output = pd.DataFrame()

    for chunk in df:
        details = chunk[cols]
    
        display(details.head())

        # output = output.append(details, igore_index=True)


In [ ]:
cols = ["din_instante", "id_subsistema", "val_geracao", "nom_tipousina"]

reading_chunks(dataframes[0], cols)

# 2) Preparação inicial dos dados

### 2.1) Seleção dos dados

# 3) Analise estatística

# 4) Pré-processamento dos dados

# 5) Modelo de predição

### 5.1) Divisão dos dados

### 5.2) Arquitetando rede LTMS